<a href="https://colab.research.google.com/github/NGeorggin/COMPSCI175FinalProject/blob/main/COMPSCI_175_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#COMPSCI 175 Final Project. Project Group 7.

##Introduction. TODO

##Libraries and Imports TODO

Import the necessary RLCard library agents and functions.

In [16]:
!pip install rlcard
import rlcard
from rlcard.agents import RandomAgent
from rlcard.utils import (
    get_device,
    set_seed,
    tournament,
    reorganize,
    Logger,
    plot_curve,
)

# TODO Cite HW2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import os
import argparse
import rlcard
from rlcard.agents import (
    CFRAgent,
    RandomAgent,
)
from rlcard.utils import (
    set_seed,
    tournament,
    Logger,
    plot_curve,
)

import json
import os
import numpy as np
from collections import OrderedDict
import csv

import rlcard
from rlcard.envs import Env
from rlcard.games.nolimitholdem import Game
from rlcard.games.nolimitholdem.round import Action


from enum import Enum

import numpy as np
from copy import deepcopy
from rlcard.games.limitholdem import Game
from rlcard.games.limitholdem import PlayerStatus

from rlcard.games.nolimitholdem import Dealer
from rlcard.games.nolimitholdem import Player
from rlcard.games.nolimitholdem import Judger
from rlcard.games.nolimitholdem import Round, Action



from rlcard.games.base import Card



from rlcard.games.limitholdem.utils import compare_hands
import numpy as np


from rlcard.games.nolimitholdem.dealer import NolimitholdemDealer

from enum import Enum

from rlcard.games.limitholdem import PlayerStatus

###

import os
import argparse
import rlcard
from rlcard.agents import (
    CFRAgent,
    RandomAgent,
)
from rlcard.utils import (
    set_seed,
    tournament,
    Logger,
    plot_curve,
)

import os
import argparse

import torch

import rlcard
from rlcard.agents import RandomAgent
from rlcard.utils import (
    get_device,
    set_seed,
    tournament,
    reorganize,
    Logger,
    plot_curve,
)



##Neural Network Construction (For Guessing Opponent's Hand)

In [17]:
class NNPolicy(nn.Module): # TODO Cite HW2
    def __init__(self, hidden):
        super(NNPolicy, self).__init__()

        # Input.
        # Community Cards (5 input nodes), Opponent's Money they've Bet (1 input node), Opponent's Last Move (1 input node).
        # 7 input nodes in total.

        self.fc1 = nn.Conv1d(1, hidden, 1)
        self.fc2 = nn.Conv1d(hidden, hidden, 1)
        self.fc3 = nn.Conv1d(hidden, 10, 7)

        # Output.
        # Array of 10 probabilities that the opponent has a particular hand.
        # 10 possible hands are: high card, pair, two pair, three of a kind, straight, flush, full house, four of a kind, straight flush, royal flush.


    def probOpponentHand(self, communityCards, oppMoney, oppLastMove, device):

        while len(communityCards) < 5:
          communityCards.append(-1) # Denotes a card slot that has not been revealed yet

        x = communityCards[:]
        x.append(oppMoney)
        x.append(oppLastMove)


        x = torch.tensor(x).float().unsqueeze(0)


        # TODO is this how you want it?
        x = self.fc1(x) # Applies Layer 1 to the values
        x = F.relu(x) # Applies ReLU filter to the values
        x = self.fc2(x) # Applies Layer 2 to the values
        x = F.relu(x) # Applies ReLU filter to the values
        x = self.fc3(x) # Applies Layer 3 to the values

        final = []

        for e in x:
          final.append(e)

        final = torch.tensor(final).float().unsqueeze(0).to(device)

        # We output the softmax
        return F.softmax(final, dim=1)

    def getAction(self, communityCards, oppMoney, oppLastMove, device):
        probs = self.probOpponentHand(communityCards, oppMoney, oppLastMove, device)#.cpu()
        m = Categorical(probs)
        # print(probs)
        action = m.sample()
        return action.item(), m.log_prob(action) # Action is some integer E [0, 9]


##Reinforcement Algorithm for the Neural Network

In [18]:
def cardSTOI(listCards):
    """
    TODO Cite
    https://rlcard.org/games.html#limit-texas-hold-em
    """

    card2index = dict()
    num = 0
    for s in ['S', 'H', 'D', 'C']:
      for r in ['A','2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q','K']:
        card2index[s+r] = num
        num += 1


    newList = []
    for c in listCards:
      newList.append(card2index[c])
    return newList

def evaluateHandType(communityCards, twoCards):
    """
    Hands labeled ...
      0. High Card
      1. Pair
      2. Two Pair
      3. Three Of A Kind
      4. Straight
      5. Flush
      6. Full House
      7. Four Of A Kind
      8. Straight Flush
      9. Royal Flush
    """

    bigCardList = communityCards[:] + twoCards[:]

    # print(bigCardList)

    if len(bigCardList) < 5:
        return -1

    suits = {'S':0 , 'C':0 , 'H':0 , 'D':0}
    ranks = {'2':0, '3':0, '4':0, '5':0, '6':0, '7':0, '8':0, '9':0, 'T':0, 'J':0, 'Q':0,'K':0,'A':0}

    for c in bigCardList:
        suits[c[0]] += 1
        ranks[c[1]] += 1

    stringOfRanks = ""
    for v in list(ranks.values()):
        if v == 0:
          stringOfRanks += "0"
        else:
          stringOfRanks += "1"


    if "11111" in stringOfRanks and (5 in list(suits.values()) or 6 in list(suits.values()) or 7 in list(suits.values())) and stringOfRanks.endswith("11111"): ## Royal Flush
        return 9
    elif "11111" in stringOfRanks and (5 in list(suits.values()) or 6 in list(suits.values()) or 7 in list(suits.values())): ## Straight Flush
        return 8
    elif 4 in list(ranks.values()): ## Four of a Kind
        return 7
    elif (3 in list(ranks.values()) and 2 in list(ranks.values())) or list(ranks.values()).count(3) >= 2:## Full House
        return 6
    elif 5 in list(suits.values()) or 6 in list(suits.values()) or 7 in list(suits.values()): ## Flush
        return 5
    elif "11111" in stringOfRanks: ## Straight
        return 4
    elif 3 in list(ranks.values()): ## Three Of A Kind
        return 3
    elif list(ranks.values()).count(2) >= 2: ## Two Pair
        return 2
    elif 2 in list(ranks.values()): ## Pair
        return 1
    else: ## High Card
        return 0

## Combining Implementation of both models

In [19]:
def NNAffectPayoff(policy, optimizer, trajectories, payoffs, device):

    agentHand = trajectories[0][-1]['raw_obs']['hand']
    communityCards = trajectories[0][-1]['raw_obs']['public_cards']
    __, oppMoney = trajectories[0][-1]['raw_obs']['all_chips']
    oppHandActual = trajectories[-1][-1]['raw_obs']['hand']

    oppLastMove = -1
    ## TODO find opp last move

    DQNExistingReward = abs(payoffs[0])
    rewardAdjustment = 0.1 * DQNExistingReward

    oppHandType, oppLogProb = policy.getAction(cardSTOI(communityCards), oppMoney, oppLastMove, device)


    agentHandType = evaluateHandType(communityCards, agentHand)

    if agentHandType == oppHandType or oppHandType == -1 or agentHandType == -1:
        return payoffs

    elif agentHandType > oppHandType:
        payoffs[0] += rewardAdjustment
        payoffs[1] -= rewardAdjustment

    elif agentHandType < oppHandType:
        payoffs[0] -= rewardAdjustment
        payoffs[1] += rewardAdjustment


    nnReturnProbs = policy.probOpponentHand(cardSTOI(communityCards), oppMoney, oppLastMove, device)#.cpu()

    # print(oppHandType)
    # print(len(list(nnReturnProbs)[0]))
    rewardArray = [0] * len(list(nnReturnProbs)[0])
    rewardArray[oppHandType] = 2 * rewardAdjustment

    # print(nnReturnProbs)

    # if oppHandActual == oppHandType:
    #     nnReturnProbs[oppHandType] = 2*nnReturnProbs[oppHandType]

    policyLoss = []
    for i in range(len(nnReturnProbs)):
        policyLoss.append(1 * (1-nnReturnProbs[0][i]) * rewardArray[i])
    policyLoss = torch.tensor(policyLoss, requires_grad=True).sum() # TODO Cite https://discuss.pytorch.org/t/runtimeerror-element-0-of-variables-does-not-require-grad-and-does-not-have-a-grad-fn/11074


    # print(f"Policy Loss: {policyLoss}")

    optimizer.zero_grad()
    policyLoss.backward()
    optimizer.step()


    return payoffs

##DQN TODO

In [20]:
def DQNtrain(args):

    # Check whether gpu is available
    device = get_device()

    # Seed numpy, torch, random
    set_seed(args.seed)

    # Make the environment with seed
    env = rlcard.make(
        args.env,
        config={
            'seed': args.seed,
        }
    )



    # Initialize the agent and use random agents as opponents
    from rlcard.agents import DQNAgent
    agent = DQNAgent(
        num_actions=env.num_actions,
        state_shape=env.state_shape[0],
        mlp_layers=[64,128],
        device=device,
        batch_size=64
    )

    games_won = 0
    csv_path2 = 'experiments/no_limit_holdem_dqn_result/result.csv'
    with open(csv_path2, 'a') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(['episode', 'reward'])

    agents = [agent]
    for _ in range(1, env.num_players):
        agents.append(RandomAgent(num_actions=env.num_actions))
    env.set_agents(agents)

    # agent = torch.load(args.save_path) ## TODO put back

    # Start training

    nn = NNPolicy(256)
    optimizer = optim.Adam(nn.parameters(), lr=0.001)

    with Logger(args.log_dir) as logger:
        for episode in range(args.num_episodes):

            # Generate data from the environment
            trajectories, payoffs = env.run(is_training=True)

            ## REINFORCE

            payoffs = NNAffectPayoff(nn, optimizer, trajectories, payoffs, device)

            # Reorganaize the data to be state, action, reward, next_state, done
            trajectories = reorganize(trajectories, payoffs)


            # Feed transitions into agent memory, and train the agent
            # Here, we assume that DQN always plays the first position
            # and the other players play randomly (if any)
            for ts in trajectories[0]:
                agent.feed(ts)

            # Evaluate the performance. Play with random agents.
            if episode % args.evaluate_every == 0:
                logger.log_performance(
                    episode,
                    tournament(
                        env,
                        args.num_eval_games,
                    )[0]
                )
                with open(csv_path2, 'a') as f:
                    csv_writer = csv.writer(f)
                    if episode == 0:
                        win_rate = 0
                    else:
                        win_rate = games_won / episode
                    csv_writer.writerow([episode, win_rate])

        # Get the paths
        csv_path, fig_path = logger.csv_path, logger.fig_path
        fig_path2 = 'experiments/no_limit_holdem_dqn_result/fig2.png'
        plot_curve(csv_path2, fig_path2, 'win_rate')

    # Plot the learning curve
    plot_curve(csv_path, fig_path, args.algorithm)

    # Save model ## TODO put back
    save_path = os.path.join(args.log_dir, 'model.pth')
    torch.save(agent, save_path)
    #agent.save_checkpoint('/content/experiments/no_limit_holdem_dqn_result/')
    print('Model saved in', save_path)

In [21]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser("DQN example in RLCard")
    parser.add_argument(
        '--env',
        type=str,
        default='no-limit-holdem',
        choices=[
            'no-limit-holdem',
        ],
    )
    parser.add_argument(
        '--algorithm',
        type=str,
        default='dqn',
        choices=[
            'dqn',
        ],
    )
    parser.add_argument(
        '--cuda',
        type=str,
        default='',
    )
    parser.add_argument(
        '--seed',
        type=int,
        default=42,
    )
    parser.add_argument(
        '--num_episodes',
        type=int,
        default=2000,
    )
    parser.add_argument(
        '--num_eval_games',
        type=int,
        default=5000,
    )
    parser.add_argument(
        '--evaluate_every',
        type=int,
        default=100,
    )
    parser.add_argument(
        '--log_dir',
        type=str,
        default='experiments/no_limit_holdem_dqn_result/',
    )
    parser.add_argument(
        '--save_path',
        type=str,
        default=os.path.join('experiments/no_limit_holdem_dqn_result/', 'model.pth'),
    )

    args, unknown = parser.parse_known_args()

    os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda
    DQNtrain(args)

--> Running on the CPU

----------------------------------------
  episode      |  0
  reward       |  -7.07894
----------------------------------------
INFO - Step 100, rl-loss: 888.2239990234375
INFO - Copied model parameters to target network.
INFO - Step 165, rl-loss: 551.539306640625
----------------------------------------
  episode      |  100
  reward       |  -0.26984
----------------------------------------
INFO - Step 316, rl-loss: 1423.94384765625
----------------------------------------
  episode      |  200
  reward       |  1.40008
----------------------------------------
INFO - Step 482, rl-loss: 2073.5146484375
----------------------------------------
  episode      |  300
  reward       |  1.54948
----------------------------------------
INFO - Step 652, rl-loss: 1706.7177734375
----------------------------------------
  episode      |  400
  reward       |  1.82066
----------------------------------------
INFO - Step 818, rl-loss: 759.9742431640625
------------------

KeyboardInterrupt: 

##Anything else that we need to include TODO